In [ ]:

import cv2
import matplotlib.pyplot as plt
img = cv2.imread('7b8902b02454db97903a146bc9e2121c.jpg', cv2.IMREAD_COLOR)[..., ::-1]
h,w,_ = img.shape

restored = cv2.resize(cv2.resize(img, (int(h/4), int(h/4))), (w,h), interpolation = cv2.INTER_NEAREST)
plt.imshow(restored)

cv2.imwrite("output.jpg", restored[..., ::-1])

In [ ]:
import torch

grid_i, grid_j = torch.meshgrid(
        torch.arange(-2, 4 + 2),
        torch.arange(-1, 4 + 1), 
        indexing='ij'
    )
grid_coord = torch.cat((grid_i.unsqueeze(2), grid_j.unsqueeze(2)), 2).float() # w,h,2
grid_i

In [ ]:
grid_coord_im2col = torch.as_strided(grid_coord, 
                size = (4, 4, 3,3, 2),
                stride=(grid_coord.stride(0) * 1, grid_coord.stride(1) * 1, 
                        grid_coord.stride(0) * 1, grid_coord.stride(1) * 1, 
                        grid_coord.stride(2)) 
    )
grid_coord_im2col[0,1]

In [ ]:
from src.modules.deform_conv import DeformConv2d, SeparableDeformConv2d
import torch 
conv = SeparableDeformConv2d(16,96, 3, padding= 1, groups= 16)
# This 2 conv_offset have the same number of parameter, but their output is different
# All output channels will use the same offset map aggregate from all input channels
conv_offset = torch.nn.Conv2d(16, 2 * 3 * 3 * 1, 3,1, 1, groups= 1) 
# Each output channel will use different offset map based on coresponding input channel
conv_offset = torch.nn.Conv2d(16, 2 * 3 * 3 * 16, 3,1, 1, groups= 16)
x = torch.rand(1,16,32,32)
offset =conv_offset(x)

out = conv(x, offset)
out.shape


In [1]:
from src.modules.flow.nested_flow import NestedAffineCoupling, InvertibleModuleWrapper, AffineCoupling, InvertibleSequential

mod = InvertibleModuleWrapper(NestedAffineCoupling(4, 3, level=2, max_stack_level= 2, enforce_channel_on_base= True), disable= True)

In [2]:
import torch
x = torch.rand(1,4,32,32)
cond = torch.rand(1,3,32,32)
# ((mod.inverse(mod(x, cond), cond) - x).abs() < 1e-7).all()
((mod.inverse(mod(x, cond), cond) - x).abs()).mean()
# mod(x, cond).shape

tensor(0.4910, grad_fn=<MeanBackward0>)

In [3]:
mod2 = InvertibleModuleWrapper(InvertibleSequential(InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'channel'), disable = True),
                            InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'channel'), disable = True),
                            InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'channel'), disable = True),
                            InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'channel'), disable = True),
                            # InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'spatial')),
                            ), disable = True)

x2 = torch.rand(1,64,32,32)
((mod2.inverse(mod2(x2)) - x2).abs()).mean()

tensor(0., grad_fn=<MeanBackward0>)

In [14]:
from torchinfo import summary
mod = InvertibleModuleWrapper(NestedAffineCoupling(64, condition_channels=32, level=2, max_stack_level= 3, 
                                                   constrain_stack = True, custom_masking = None), disable= True)
mod._fn.check_total_flow()
summary(mod,[(1,64,128,128), (1,32,128,128)], depth= 3)

5


Layer (type:depth-idx)                                            Output Shape              Param #
InvertibleModuleWrapper                                           [1, 64, 128, 128]         --
├─NestedAffineCoupling: 1-1                                       [1, 64, 128, 128]         --
│    └─ModuleList: 2-11                                           --                        (recursive)
│    │    └─ConditionalModule: 3-1                                [1, 32, 128, 128]         2,368
│    └─ModuleList: 2-10                                           --                        (recursive)
│    │    └─NestedAffineCoupling: 3-2                             [1, 32, 128, 128]         332,672
│    └─ModuleList: 2-11                                           --                        (recursive)
│    │    └─ConditionalModule: 3-3                                [1, 32, 128, 128]         (recursive)
│    └─ModuleList: 2-12                                           --                        (rec

In [17]:
mod2 = InvertibleModuleWrapper(InvertibleSequential(
                            InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'channel')),
                            InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'spatial')),
                            InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'channel')),
                            InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'spatial')),
                            InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'channel')),
                            # InvertibleModuleWrapper(AffineCoupling(in_channels=64, masking_type= 'spatial')),
                            ))
summary(mod2,[(1,64,128,128)], depth= 3)

Layer (type:depth-idx)                                  Output Shape              Param #
InvertibleModuleWrapper                                 [1, 64, 128, 128]         --
├─InvertibleSequential: 1-1                             --                        --
│    └─InvertibleModuleWrapper: 2-1                     [1, 64, 128, 128]         --
│    │    └─AffineCoupling: 3-1                         --                        75,904
│    └─InvertibleModuleWrapper: 2-2                     [1, 64, 128, 128]         --
│    │    └─AffineCoupling: 3-2                         --                        303,360
│    └─InvertibleModuleWrapper: 2-3                     [1, 64, 128, 128]         --
│    │    └─AffineCoupling: 3-3                         --                        75,904
│    └─InvertibleModuleWrapper: 2-4                     [1, 64, 128, 128]         --
│    │    └─AffineCoupling: 3-4                         --                        303,360
│    └─InvertibleModuleWrapper: 2-5       

In [27]:
torch.linspace(1,2,4).int()

tensor([1, 1, 1, 2], dtype=torch.int32)

In [22]:
from multiprocessing import Process
import time
from typing import List
import numpy as np
import torch
import torch.nn as nn
import torch.multiprocessing as mp

class MLP(nn.Module):
    def __init__(self, network_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(2, network_size)
        self.fc2 = nn.Linear(network_size, 1)

    def forward(self, x):
        return torch.sigmoid(self.fc2(self.fc1(x)))
    
def inner_train(model, optimizer, num_iterations, criterion, data, targets):
    for _ in range(num_iterations):
        output = model(data) 
        loss = criterion(output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
def train(num_networks, network_size, num_iterations):
    training_start = time.perf_counter()
    criterion = torch.nn.BCELoss()
    data = torch.zeros((5, 2), device='cuda')
    targets = torch.ones((5, 1), device='cuda')
    
    models = []
    optimizers = []
    for _ in range(num_networks):
        model = MLP(network_size).cuda()
        models.append(model)

        optimizer = torch.optim.Adam(model.parameters())
        optimizers.append(optimizer)

    num_processes = num_networks
    processes: List[Process] = []
    for model, optimizer in zip(models, optimizers):
        p = mp.Process(target=inner_train, args=(model, optimizer, num_iterations, criterion, data, targets,))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()
    
    print(f"Training {num_networks} model took {time.perf_counter() - training_start:.2f}s")
    
def train2(num_networks, network_size, num_iterations):
    training_start = time.perf_counter()
    criterion = torch.nn.BCELoss()
    data = torch.zeros((5, 2), device='cuda')
    targets = torch.ones((5, 1), device='cuda')
    
    models = []
    for _ in range(num_networks):
        models.append(MLP(network_size).cuda())
    for model in models:
        optimizer = torch.optim.Adam(model.parameters())
        for _ in range(num_iterations):
            output = model(data)
            loss = criterion(output, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    print(f"Training {num_networks} model took {time.perf_counter() - training_start:.2f}s")


train(1, 20, 1000)

train(5, 20, 1000)

train(15, 20, 1000)


Training 1 model took 0.08s
Training 5 model took 0.29s
Training 15 model took 0.87s
